In [1]:
!pip install -q transformers

     |████████████████████████████████| 4.2 MB 37.5 MB/s 
     |████████████████████████████████| 596 kB 54.4 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 346 kB 15.5 MB/s 
     |████████████████████████████████| 212 kB 46.3 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 58.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.9 MB/s 
     |████████████████████████████████| 127 kB 57.7 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 41.6 MB/s 
     |████████████████████████████████| 271 kB 42.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install -q huggingface_hub

In [4]:
!pip search espeak
!pip install py-espeak-ng
!sudo apt-get install python-espeak
!sudo apt-get update && sudo apt-get install espeak

ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI's XMLRPC API is currently disabled due to unmanageable load and will be deprecated in the near future. See https://status.python.org/ for more information.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0 python-espeak
0 upgraded, 5 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,166 kB of archives.
After this operation, 2,859 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak

In [5]:
!pip install -q phonemizer

     |████████████████████████████████| 90 kB 6.7 MB/s 
     |████████████████████████████████| 197 kB 66.0 MB/s 
     |████████████████████████████████| 41 kB 408 kB/s 


In [6]:
import soundfile as sf
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
import re
import zipfile
import pandas as pd
import phonemizer
import random
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2Tokenizer, Wav2Vec2Config
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft", output_hidden_states=True)
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft", config=config)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/412 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

### Первый этап: сбор данных

В датасете TIMIT для каждой аудиозаписи можно найти ещё три файла: 
- .phn - файл, в котором на каждой строке выписаны начало и конец произнесения звука и сам звук
- .wrd - файл, в котором на каждой строке выписаны начало и конец произнесения слова и само слово
- .wav - файл аудиозаписи

In [9]:
# importing necessary modules
import requests, zipfile
from io import BytesIO
print('Downloading started')

#Defining the zip file URL
url = 'https://data.deepai.org/timit.zip'

# Split URL to get the file name
filename = url.split('/')[-1]

# Downloading the file by sending the request to the URL
req = requests.get(url)
print('Downloading Completed')

# extracting the zip file contents
my_zipfile= zipfile.ZipFile(BytesIO(req.content))
#zipfile.extractall()

In [10]:
needed_files = [file for file in my_zipfile.namelist() if file.endswith('.wav')]

В переменной *vocab* хранится словарь, где каждому звуку соответствует его индекс в векторе. Так как для дальнейшей работы удобнее сделать так, чтобы ключом был индекс, а значением звук, что сохранено в переменной *convenient_vocab*.

In [11]:
_file = hf_hub_download(repo_id="facebook/wav2vec2-xlsr-53-espeak-cv-ft", filename="vocab.json")
with open(_file, 'r') as f:
  vocab = json.loads(f.read())

Downloading:   0%|          | 0.00/4.64k [00:00<?, ?B/s]

In [12]:
convenient_vocab = {}
for k, v in vocab.items():
  convenient_vocab[v] = k

Следующие две функции создают списки, в которых можно найти:
1. Начало и конец звучания слова
2. Само слово
3. Следующее после него слово.

Функция *find_s* ищет все слова, которые оканчиваются на букву *s*, не являющейся клитикой посессивности.

Функция *find_t* ищет все слова, которые оканчиваются на -ed или имеют звук [t].

In [13]:
def find_phoneme(filename):
  with my_zipfile.open(filename, 'r') as f:
    lines = f.readlines()
    phonemes_words = []
    for i in range(len(lines)-1):
      bgn, end, word = lines[i].split() 
      next_word = lines[i+1].split()[-1].decode("utf-8")
      bgn = int(bgn.decode("utf-8"))
      end = int(end.decode("utf-8"))
      word = word.decode("utf-8")
      if word.endswith('s') and "'" not in word:
        phonemes_words.append([bgn, end, word, next_word])
    return phonemes_words

In [14]:
def find_new_phoneme(filename):
  with my_zipfile.open(filename, 'r') as f:
    lines = f.readlines()
    phonemes_words = []
    for i in range(len(lines)-1):
      bgn, end, word = lines[i].split()
      next_word = lines[i+1].split()[-1].decode("utf-8")
      bgn = int(bgn.decode("utf-8"))
      end = int(end.decode("utf-8"))
      word = word.decode("utf-8")
      ind = word.find('t')
      if ((ind != -1) and (ind != len(word)-1)) or (word.endswith('ed')):
        phonemes_words.append([bgn, end, word, next_word])
    return phonemes_words

Следующие две ячейки собирают данные для каждого из тестов на фонемы. В каждом из них проверяется, чтобы не происходило частичных ассимиляций по звонкости.

Для данных на фонемы /t/ и /d/ идёт проверка на то, чтобы в написании слова не встречались буквы s и t рядом, так как подобное сочетание часто прочитывается так, что влияет на реализацию [t].

In [15]:
all_words_arrays = []
for filename in tqdm(needed_files):
  if '.wav' in filename:
    with my_zipfile.open(filename) as f:
      audio, sample_rate = sf.read(f)
      audio = audio.astype(np.float64)
      audio = audio / np.max(np.abs(audio))
    wrd_file=filename.replace(".WAV.wav",".WRD")
    wrd_sig=find_phoneme(wrd_file)
    for word in wrd_sig:
      if (word[3][0] in ['p', 't', 'h', 'c', 'f', 'k', 's']) and \
      (word[2][-2] in ['p', 't', 'h', 'c', 'f', 'k', 's', 'a', 'e', 'u', 'o', 'i']):
        word_phoneme = {
            word[2]: {
                'audio': audio[word[0]:word[1]], 
                'sample_rate': sample_rate
                }
              }
        all_words_arrays.append(word_phoneme)

100%|██████████| 6300/6300 [00:28<00:00, 223.73it/s]


In [16]:
all_words_arrays_t = []
for filename in tqdm(needed_files):
  if '.wav' in filename:
    with my_zipfile.open(filename) as f:
      audio, sample_rate = sf.read(f)
      audio = audio.astype(np.float64)
      audio = audio / np.max(np.abs(audio))
    wrd_file=filename.replace(".WAV.wav",".WRD")
    wrd_sig=find_new_phoneme(wrd_file)
    for word in wrd_sig:
      ind = word[2].find('t')
      if ind != -1:
        if (word[2][ind-1] == 's' and ind != 0) or (word[2][ind+1] == 's'):
          word_phoneme = {
            word[2]: {
                'audio': audio[word[0]:word[1]], 
                'sample_rate': sample_rate
                }
              }
          all_words_arrays_t.append(word_phoneme)
      if (word[2].endswith('ed')) and (word[2][-3] in ['p', 't', 'h', 'c', 'f', 'k', 's']):
        word_phoneme = {
            word[2]: {
                'audio': audio[word[0]:word[1]], 
                'sample_rate': sample_rate
                }
              }
        all_words_arrays_t.append(word_phoneme)

100%|██████████| 6300/6300 [00:39<00:00, 158.49it/s]


Функция *all_vectors* записывает вектора звуков, а также слова, в которых встретились нужные звуки. Переменная *testname* контролирует тест, для которого на данный момент собираются данные.

In [60]:
def all_vectors(n, layer, predicted_ids, word, testname): #, next_word):
  numpy_logits = layer[0].detach().numpy()
  for i in range(len(predicted_ids[0])):
    if predicted_ids[0][i] not in [1, 2, 3, 0]:
      all_sound_vectors[testname][n][convenient_vocab[predicted_ids[0][i]]].append(numpy_logits[i])
    if n == 0:
      if testname == 'z':
        if (convenient_vocab[predicted_ids[0][i]] == 's') or \
                    (convenient_vocab[predicted_ids[0][i]] == 'z'):
          words[convenient_vocab[predicted_ids[0][i]]].append(word)
    elif testname == 't':
      if (convenient_vocab[predicted_ids[0][i]] == 't') or \
                  (convenient_vocab[predicted_ids[0][i]] == 'd'):
        words_t[convenient_vocab[predicted_ids[0][i]]].append(word)

In [61]:
all_sound_vectors = {'z': [defaultdict(list) for i in range(26)], 't': [defaultdict(list) for i in range(26)]}
all_near_sounds = defaultdict(list)
words_t = {'t': [], 'd': []}
words = {'z': [], 's': []}
layers = []
all_keys = []

In [62]:
testname = 'z'  # 'z' or 't'
for testname in ['t', 'z']:
  if testname == 'z':
    arrays = all_words_arrays
  else:
    arrays = all_words_arrays_t
  for audio in tqdm(arrays):
    key = list(audio.keys())[0]
    all_keys.append(key)
    audio_input = audio[key]['audio']
    sample_rate = audio[key]['sample_rate']
    input_values = processor(audio_input, 
                            sampling_rate=sample_rate, 
                            return_tensors="pt").input_values
    with torch.no_grad():
      outputs = model(input_values.float())
      logits = outputs.logits
      hidden_states = outputs.hidden_states
      layers.extend(hidden_states)
      layers.append(logits)
    predicted_ids = torch.argmax(logits, dim=-1).detach().numpy()
    for n, layer in enumerate(layers):
      all_vectors(n, layer, predicted_ids, key, testname)
    layers = []

100%|██████████| 1196/1196 [08:50<00:00,  2.25it/s]


In [63]:
nlp = spacy.load("en_core_web_sm")

### Второй этап: обработка данных и тесты

- *is_plural_wn* получает на вход слово и определяет часть речи и, если слово - существительное, число. Значение 'hello' используется для слов, которые не подходят для теста.

- *is_past* получает на вход слово и определяет, является ли полученное слово глаголом в прошедшем времени.

- *count_accuracy* получает на вход слова, вектора нужных звуков, которые входят в соответствующие слова и список звуков. Внутри функции создаётся таблица, где в каждоей строке есть вектор звука, слово и обозначение звука. Дальше применяется описанные функции и очищаются данные. После на этих данных считается метрика accuracy в двух случаях: на верных данных и на перемешанных.

In [64]:
def is_plural_wn(noun):
  word = nlp(noun)
  mistakes = ['is', 'was', 'has', 
              'this', 'as', 'his', 
              'does', 'themselves', 'requires', 
              'contains']
  if str(word[0]) in mistakes:
    return 'hello'
  if word[0].pos_ == 'NOUN':
    wnl = WordNetLemmatizer()
    lemma = wnl.lemmatize(noun, 'n')
    plural = 1 if noun is not lemma else 0
    return plural
  else:
     return 'hello'

In [65]:
def is_past(word):
  nlp_word = nlp(word)
  if nlp_word[0].pos_ == 'VERB':
    wnl = WordNetLemmatizer()
    lemma = wnl.lemmatize(word, 'v')
    if word != lemma:
      if word.endswith('ed'):
        return 1
    return 0
  else:
     return 0

In [66]:
def count_accuracy(words, emb, sounds, layer):
  random.seed(333)
  data = pd.DataFrame({'word': words, 'vector_emb': emb, 'sound': sounds})
  if 's' in sounds:
    func = is_plural_wn
    data['is'] = data['word'].apply(func)
    res = data[data['is']!='hello'].copy()
    del_words = ['solves', 'tastes', 'celebrates', 
                 'misperceives', 'chases', 'decides', 
                 'involves', 'presumes', 'presents', 
                 'guarantees', 'sets', 'serves', 
                 'causes', 'increases', 'preserves', 
                 'slips', 'prints']
    res = res[~res['word'].isin(del_words)]
  elif 't' in sounds:
    mistakes = ['mopped', 'slanted', 'squeaked', 'invoked', 
                'shocked', 'unfenced', 'crooked', 'uninterrupted', 
                'melted', 'pumped', 'engulfed', 'advanced', 
                'unabashed', 'addressed', 'tilted', 'chorused', 'polished']
    func = is_past
    data['is'] = data['word'].apply(func)
    res = data.copy()
    for m in mistakes:
      res.loc[res['word']==m, 'is_past'] = 1
  answers = [i for i in res['is'].tolist()]
  random.shuffle(answers)

  model_log = LogisticRegression(max_iter=10000)
  X_train, X_test, y_train, y_test = train_test_split(
                                      res['vector_emb'].tolist(), 
                                      res['is'].tolist(), 
                                      train_size = .8, 
                                      random_state=10)
  under_sampler = RandomUnderSampler(random_state=42)
  X_res, y_res = under_sampler.fit_resample(X_train, y_train)
  model_log.fit(X_res, y_res)
  score = model_log.score(X_test, y_test)
  
  model_rn = LogisticRegression(max_iter=10000)
  X_train, X_test, y_train, y_test = train_test_split(
                                      res['vector_emb'].tolist(), 
                                      answers, 
                                      train_size = .8, 
                                      random_state=10)
  under_sampler = RandomUnderSampler(random_state=42)
  X_res, y_res = under_sampler.fit_resample(X_train, y_train)
  model_log.fit(X_res, y_res)
  rn_score = model_log.score(X_test, y_test)

  means = None
  if layer == 25:
    means = res.groupby('is')['vector_emb'].agg(np.mean).reset_index()

  return score, rn_score, model_log.coef_[0], means

In [67]:
scores = {'t': [], 'z': []}
rn_scores = {'t': [], 'z': []}
coefs = {'t': [], 'z': []}
means = {'t': [], 'z': []}

In [73]:
len(words_t['t']) / 25

1474.0

In [76]:
for testname in ['t', 'z']:
  if testname == 't':
    all_words = words_t['t'][:1474]
    all_sounds = ['t']*len(all_words)
  elif testname == 'z':
    s_words = words['s']
    z_words = words['z']
    all_words = s_words + z_words
    s_sounds = ['s']*len(s_words)
    z_sounds = ['z']*len(z_words)
    all_sounds = s_sounds + z_sounds
  for i in tqdm(range(26)):
    if testname == 't':
      all_emb = all_sound_vectors[testname][i]['t']
    elif testname == 'z':
      all_emb = all_sound_vectors[testname][-1]['s'] + all_sound_vectors[testname][-1]['z']
    score, rn_score, coef, mean = count_accuracy(all_words, all_emb, all_sounds, i)
    scores[testname].append(score)
    rn_scores[testname].append(rn_score)
    coefs[testname].append(coef)
    if mean:
      means[testname].append(mean)

100%|██████████| 1/1 [00:07<00:00,  7.46s/it]


In [80]:
scores['t'] = scores['t'][:-1]

In [83]:
mean

In [ ]:
means['t'].append(means)

### Третий этап: визуализация

Функция для изображения данных:

- draw_coefs рисует графики наибольших по модулю коэффициентов модели.
- draw_accuracy рисует график значений accuracy на каждом слое.


In [55]:
def draw_coefs(coefs, testname):
  '''
  model -> используемая модель LogReg
  testname -> название теста
  '''
  all_coefs = np.argsort(coefs)
  need_coefs = all_coefs[:10].tolist() + all_coefs[-10:].tolist()
  sounds = []
  coefs = []
  for coef in need_coefs:
    sounds.append(convenient_vocab[coef])
    coefs.append(model.coef_[0][coef])
  plt.figure(figsize=(7, 5))
  plt.bar(sounds, coefs)
  plt.xlabel('Звуки')
  plt.ylabel('Коэффициенты')
  plt.title(f'Звуки с наибольшими и наименьшими коэффициентами\nлогистической регрессии, тест: {testname}')

In [ ]:
def draw_accuracy(dict_of_scores):
  '''
  dict_of_scores -> словарь, в котором ключом является тип перемешивания,
  а значением список значений accuracy
  '''
  xs = [i for i in range(1, 27)]
  for k, v in dict_of_scores.items():
    plt.plot(xs, v, label=k)
  plt.legend()
  plt.title('Accuracy предсказания звонкости звука на каждом из слоёв')
  plt.xlabel('Номер слоя')
  plt.ylabel('Значение accuracy')

In [ ]:
means = res.groupby('is_past')['vector_emb'].agg(np.mean).reset_index()

In [ ]:
for i, row in means.iterrows():
  sort_means = row['vector_emb'][needed_coefs]
  print(sort_means)
  #print([convenient_vocab[index] for index in sort_means[-1:-10:-1]])

[-1.1774392   0.82095844 -1.7124165  -2.8348618  -3.797132   -3.4806006
 -5.1169715  -2.9093437  -5.4074383  -2.3262134 ]
[-0.8332333   0.83449155 -1.6074705  -2.795156   -3.9942548  -3.687091
 -5.1707253  -3.0722408  -5.489696   -2.1666198 ]


In [ ]:
s_and_z = {'s': [], 'z': []}
for sound in ['s', 'z']:
  s_and_z[sound] = all_sound_vectors[-1][sound]

In [ ]:
s = list(s_and_z.values())[0]
z = list(s_and_z.values())[1]
all_emb = s + z
s_words = words['s']
z_words = words['z']
all_words = s_words + z_words
s_sounds = ['s']*len(s_words)
z_sounds = ['z']*len(z_words)
s_z = s_sounds + z_sounds
#s_next_words = next_words['s']
#z_next_words = next_words['z']
#all_next_words = s_next_words + z_next_words